<a href="https://colab.research.google.com/github/hwankang/Defense/blob/main/0718_taco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy scipy librosa unidecode inflect

In [ ]:
import torch 

tacotron2 = torch.hub.load('nvidia/DeepLearningExamples:torchhub','nvidia_tacotron2')
waveglow = torch.hub.load('nvidia/DeepLearningExamples:torchhub','nvidia_waveglow')

In [ ]:
tacotron2 = tacotron2.to('cuda')
tacotron2.eval()

In [ ]:
waveglow = waveglow.remove_weightnorm(waveglow)
waveglow = waveglow.to('cuda')
waveglow.eval()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')

def plot_data(data, figsize=(16,4)):
    fig,axes = plt.subplots(1, len(data), figsize=figsize)
    for i in range(len(data)):
        axes[i].imshow(data[i],aspect='auto', origin='bottom',
                       interpolation='none',cmap='viridis')

def TTS(text):
    sampling_rate = 22050
    sequence = np.array(tacotron2.text_to_sequence(text,['english_cleaners']))[None,:]
    sequence = torch.from_numpy(sequence).to(device='cuda',dtype=torch.int64)

    with torch.no_grad():
        mel_outputs,mel_outputs_postnet, _, alignments=tacotron2.infer(sequence)
        audio= waveglow.infer(mel_outputs_postnet)

    mel_output = mel_outputs.data.cpu().numpy()[0]
    mel_output_postnet = mel_ouputs_postnet.data.cpu().numpy()[0]
    alignment=alignments.data().cpu().numpy()[0].T
    audio_np=audio[0].data.cpu().numpy()

    return mel_output, mel_output_postnet, alignment, audio_np, sampling_rate

In [ ]:
import librosa.display
from IPython.display import Audio

text="Hello, How are you?"
mel_output, mel_output_postnet, alignment, audio_np, sampling_rate = TTS(text)

fig=plt.figure(figsize=(14,4))
librosa.display.waveplot(audio_np,sr=sampling_rate)
plot_data((mel_ouput, mel_output_postnet,alignment))
Audio(audio_np,rate=sampling_rate)